In [9]:
%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

## Differentially Private SVM Algorithm Tutorial


The following tutorial gives one example of how a differentially private pipeline of PCA and SVM functions are called. The data set used is generated from randomly drawn samples from a mulitivariate Gaussian distribution. A non-differentially private pipeline of PCA and SVM is also utilized. This is used to compare the results (classification accuracy) of the differentially private pipeline.

###### A sample of the pipeline is shown below:

Generate samples --> Perform PCA for dimension reduction --> Perform SVM for classifier training --> Test classifier

The parameters that can be adjusted are:

- Epsilon_pca
- Epsilon_svm

In [10]:
from ipywidgets import interact
from IPython.display import display

# This tutorial shows one common pipeline for learning a binary classifier


# This function is used to randomly generate data samples from a multivariate Gaussian distribution.
def gen_data(num_tst_samp, k):
    """
    Inputs:
            num_tst_samp: total number of test samples to return in data matrix
            n: number of samples to generate for each class
    Outputs:
            data: data matrix with samples in rows, [nxd]
            labels: n dimensional vector
    """
    import numpy as np
    import dp_stats as dps

    d = 10    # features
    n = num_tst_samp

    # create covariance matrix
    A = np.zeros((d,d))
    for i in range(d):
        if i < k:
            A[i,i] = d - i
        else:
            A[i, i] = 1

    # create mean for class 1
    mean1 = -1 * np.ones(d)

    # create mean for class 2
    mean2 = np.ones(d)

    # generate n samples for class 1
    cls1_samps = np.random.multivariate_normal(mean1, A, n)    # [nxd]
    # generate n samples for class 2
    cls2_samps = np.random.multivariate_normal(mean2, A, n)    # [nxd]
    return cls1_samps, cls2_samps, A

# This function is used to randomly mix the two class samples and return training and testing data
def sample_selection(data_cls1, data_cls2, trn_size, N):
    """
    Inputs:
            data_cls1: class 1 data, samples are in rows [nxd]
            data_cls2: class 2 data, samples are in rows [nxd]
            trn_size: number of samples to use for training data, integer
            tst_size: number of samples to use for testing data, integer
    Outputs:
            trn_data: [trn_size x d]
            trn_labels:[trn_size]
            tst_data: [tst_size x d]
            tst_labels: [tst_size]
    """
    import numpy as np

    ind = np.random.permutation(N)
    trn_data1 = data_cls1[ind[:trn_size],:]
    trn_data2 = data_cls2[ind[:trn_size],:]
    tst_data1 = data_cls1[ind[trn_size:N],:]
    tst_data2 = data_cls2[ind[trn_size:N],:]

    trn_data = np.concatenate((trn_data1, trn_data2),axis=0)
    trn_labels = np.concatenate( (np.ones(trn_size), -1*np.ones(trn_size)),axis=0 )

    tst_data = np.concatenate((tst_data1, tst_data2),axis=0)
    tst_labels = np.concatenate((np.ones(N-trn_size),-1*np.ones(N-trn_size)),axis=0)

    return trn_data, trn_labels, tst_data, tst_labels

# this function will score the differentially private classifier
def test_dp_clf(data_tst, labels_tst, dp_clf):
    import numpy as np
    
    # loop throught the data and record wrong answers
    n = len(labels_tst)
    tot_err = 0
    
    for i in range(n):
        dp_ans = np.dot(data_tst[i,:], dp_clf)
        if labels_tst[i] < 0:
            if dp_ans >= 0:
                tot_err += 1
        else:
            if dp_ans <= 0:
                tot_err += 1
    accuracy = (n - tot_err) * 1.0 / (n * 1.0)
    return accuracy

# This function is used give the pipeline interactive control.
def svm_pipeline(Epsilon_pca = 0.5, Epsilon_svm = 0.5):
    from sklearn import svm
    import numpy as np
    import dp_stats as dps
    
    # first generate the training and testing data
    cls1, cls2, A = gen_data(num_tst_samp = 2200, k = 4)
    
    trn_data, trn_labels, tst_data, tst_labels = sample_selection(cls1, cls2, trn_size = 1000, N = 1100)
    
    # go through the non-differentially private PCA routine
    sigma_control = np.dot(trn_data.T, trn_data)     # [d x d] = [d x Sample_size] [Sample_size x d]
    U, S, V = np.linalg.svd(sigma_control)
    
    # grab the first k columns
    U_reduc = U[:, :4]      # [d x k]
    
    # project the data onto the k subspace
    trn_data_reduc = np.dot(trn_data, U_reduc)
    
    # go through SVM routine
    clf = svm.SVC(kernel = 'linear', gamma = 0.01, C = 10)
    clf.fit(trn_data_reduc, trn_labels)
    
    # reduce testing data to use to score the control classifier
    tst_data_reduc = np.dot(tst_data, U_reduc)    # [d x k]
    control_score = clf.score(tst_data_reduc, tst_labels)
    #print(control_score)
    
    # go through differentially private pipeline
    # dp_pca_sn ( data, epsilon=1.0 )  // samples must be in columns
    sigma_dp = dps.dp_pca_ag(tst_data.T, epsilon = Epsilon_pca)
    U_dp, S_dp, V_dp = np.linalg.svd(sigma_dp)
    
    # grab the first k columns
    U_dp_reduc = U_dp[:, :4]
    
    # project the data
    dp_trn_data = np.dot(trn_data, U_dp_reduc)
    
    # go through differentially private svm routine
    # dp_svm(data, labels, method='obj', epsilon=0.1, Lambda = 0.01, h = 0.5)
    clf_dp = dps.dp_svm(dp_trn_data, trn_labels, epsilon = Epsilon_svm)
    
    # reduce the testing data
    tst_dp_data = np.dot(tst_data, U_dp_reduc)
    
    # test the differentially private classifier
    dp_score = test_dp_clf(tst_dp_data, tst_labels, clf_dp)
    #print(dp_score)
    
    # output the results
    control_txt = "Non-private Quality: {}".format(round(control_score, 4))
    display(control_txt)
    dp_txt = "Differentially Private Quality: {}".format(round(dp_score, 4))
    display(dp_txt)
    
interact(svm_pipeline, Epsilon_pca=(0.01,1.0,0.05), Epsilon_svm=(0.01,1.0, 0.05))

'Non-private Quality: 0.9'

'Differentially Private Quality: 0.895'

<function __main__.svm_pipeline>

## Differentially Private LR Algorithm Tutorial


The following tutorial gives one example of how a differentially private pipeline of PCA and LR funcitons are called. The data set used is generated from randomly drawn samples from a mulitivariate Gaussian distribution. A non-differentially private pipeline of PCA and LR is also  completed. This is used to compute the accuracy of the differentially private pipeline.

###### A sample of the pipeline is shown below:

Generate samples --> Perform PCA for dimension reduction --> Perform LR for classifier training --> Test classifier

The parameters that can be adjusted are:

- Epsilon_pca
- Epsilon_svm

In [11]:
# This tutorial shows one common way to implement a logistic regression pipeline

# This funciton is used for an interactive pipeline
def lr_pipeline(Epsilon_pca = 0.5, Epsilon_svm = 0.5):
    import numpy as np
    import dp_stats as dps
    from sklearn import linear_model
    
    # go through non-differentially private pipeline first
    
    # first generate the training and testing data
    cls1, cls2, A = gen_data(num_tst_samp = 2200, k = 4)
    
    trn_data, trn_labels, tst_data, tst_labels = sample_selection(cls1, cls2, trn_size = 1000, N = 1100)
    
    # go through the non-differentially private PCA routine
    sigma_control = np.dot(trn_data.T, trn_data)     # [d x d] = [d x Sample_size] [Sample_size x d]
    U, S, V = np.linalg.svd(sigma_control)
    
    # grab the first k columns
    U_reduc = U[:, :4]      # [d x k]
    
    # project the data onto the k subspace
    trn_data_reduc = np.dot(trn_data, U_reduc)
    
    # go through logistic regression routine
    lr_control = linear_model.LogisticRegression(C=1e5)
    lr_control.fit(trn_data_reduc, trn_labels)
    
    # reduce testing data to use to score the control classifier
    tst_data_reduc = np.dot(tst_data, U_reduc)    # [d x k]
    control_score = lr_control.score(tst_data_reduc, tst_labels)
    print(control_score)
    
    # go through differentially private pipeline
    # dp_pca_sn ( data, epsilon=1.0 )  // samples must be in columns
    sigma_dp = dps.dp_pca_ag(tst_data.T, epsilon = Epsilon_pca)
    U_dp, S_dp, V_dp = np.linalg.svd(sigma_dp)
    
    # grab the first k columns
    U_dp_reduc = U_dp[:, :4]
    
    # project the data
    dp_trn_data = np.dot(trn_data, U_dp_reduc)
    
    # go through differentially private lr routine 
    # dp_lr(data, labels, method='obj', epsilon=0.1, Lambda = 0.01 )
    lr_dp = dps.dp_lr(dp_trn_data, trn_labels, epsilon = Epsilon_svm)
    
    # reduce the testing data
    tst_dp_data = np.dot(tst_data, U_dp_reduc)
    
    # test the differentially private classifier
    dp_score = test_dp_clf(tst_dp_data, tst_labels, lr_dp)
    print(dp_score)
    
    # output the results
    control_txt = "Non-private Quality: {}".format(round(control_score, 4))
    display(control_txt)
    dp_txt = "Differentially Private Quality: {}".format(round(dp_score, 4))
    display(dp_txt)

interact(lr_pipeline, Epsilon_pca=(0.01,1.0,0.05), Epsilon_svm=(0.01,1.0, 0.05))

0.885
0.91


'Non-private Quality: 0.885'

'Differentially Private Quality: 0.91'

In [12]:
%%HTML
<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Created with Jupyter, delivered by Fastly, rendered by Rackspace.
</footer>